In [0]:
!pip install http://download.pytorch.org/whl/cu90/torch-1.0.0-cp36-cp36m-linux_x86_64.whl
!pip install torchvision
!pip install tqdm


In [14]:
!pip install tensorboardcolab

In [12]:
!tensorboard --logdir /content/cnn

TensorBoard 1.12.1 at http://29bfae16dbc9:6006 (Press CTRL+C to quit)
^C


In [5]:
import torch
from torch import nn, optim
from torch.utils.data import (Dataset, DataLoader, TensorDataset)
import tqdm

from torchvision.datasets import FashionMNIST
from torchvision import transforms

# 훈려용 데이터 가져오기
# 초기 상태에선 PIL（Python Imaging Library) 이미지 형식으로
# Dataset를 만들어 버린다.
# 따라서 transforms.ToTensor를 사용해 Tensor로 변환한다
fashion_mnist_train = FashionMNIST("/content/FashionMNIST",
    train=True, download=True,
    transform=transforms.ToTensor())
# 검증용 데이터 가져오기
fashion_mnist_test = FashionMNIST("/content/FashionMNIST",
    train=False, download=True,
    transform=transforms.ToTensor())

# 배치 크기가 128인 DataLoader를 각각 작성
batch_size=128
train_loader = DataLoader(fashion_mnist_train,
                          batch_size=batch_size, shuffle=True)
test_loader = DataLoader(fashion_mnist_test,
                          batch_size=batch_size, shuffle=False)

Processing...
Done!


In [0]:
# (N, C, H, W)혀익의 Tensor를(N, C*H*W)로 늘리는 계층
# 합성곱 출력을 MLP에 전달할 때 필요
class FlattenLayer(nn.Module):
    def forward(self, x):
        sizes = x.size()
        return x.view(sizes[0], -1)

# 5×5의 커널을 사용해서 처음에 32개, 다음에 64개의 채널 작성
# BatchNorm2d는 이미지용 Batch Normalization
# Dropout2d는 이미지용 Dropout
# 마지막으로 FlattenLayer 적용
conv_net = nn.Sequential(
    nn.Conv2d(1, 32, 5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(32),
    nn.Dropout2d(0.25),
    nn.Conv2d(32, 64, 5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.Dropout2d(0.25),
    FlattenLayer()
)

# 합성곱에 의해 최종적으로 이미지 크기가 어떤지를
# 더미 데이터를 넣어서 확인한다

test_input = torch.ones(1, 1, 28, 28)
conv_output_size = conv_net(test_input).size()[-1]

# 2층 MLP
mlp = nn.Sequential(
    nn.Linear(conv_output_size, 200),
    nn.ReLU(),
    nn.BatchNorm1d(200),
    nn.Dropout(0.25),
    nn.Linear(200, 10)
)
# 최종 CNN
net = nn.Sequential(
    conv_net,
    mlp
)

In [0]:
# 평가용 헬퍼 함수
def eval_net(net, data_loader, device="cpu"):
    # Dropout 및 BatchNorm을 무효화
    net.eval()
    ys = []
    ypreds = []
    for x, y in data_loader:
        # to 메서드로 계산을 실행할 디바이스로 전송
        x = x.to(device)
        y = y.to(device)
        # 확률이 가장 큰 클래스를 예측(리스트 2.1 참조)
        # 여기선 forward（추론） 계산이 전부이므로 자동 미분에
        # 필요한 처리는 off로 설정해서 불필요한 계산을 제한다
        with torch.no_grad():
            _, y_pred = net(x).max(1)
        ys.append(y)
        ypreds.append(y_pred)
    
    # 미니 배치 단위의 예측 결과 등을 하나로 묶는다
    ys = torch.cat(ys)
    ypreds = torch.cat(ypreds)
    # 예측 정확도 계산
    acc = (ys == ypreds).float().sum() / len(ys)
    return acc.item()

# 훈련용 헬퍼 함수
def train_net(net, train_loader, test_loader,
              optimizer_cls=optim.Adam,
              loss_fn=nn.CrossEntropyLoss(),
              n_iter=10, device="cpu", writer=None):
    train_losses = []
    train_acc = []
    val_acc = []
    optimizer = optimizer_cls(net.parameters())
    for epoch in range(n_iter):
        running_loss = 0.0
        # 신경망을 훈련 모드로 설정
        net.train()
        n = 0
        n_acc = 0
        # 시간이 많이 걸리므로 tqdm을 사용해서 진행바를 표시
        for i, (xx, yy) in tqdm.tqdm(enumerate(train_loader),
            total=len(train_loader)):
            xx = xx.to(device)
            yy = yy.to(device)
            h = net(xx)
            loss = loss_fn(h, yy)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            n += len(xx)
            _, y_pred = h.max(1)
            n_acc += (yy == y_pred).float().sum().item()
        train_losses.append(running_loss / i)
        # 훈련 데이터의 예측 정확도
        train_acc.append(n_acc / n)

        # 검증 데이터의 예측 정확도
        val_acc.append(eval_net(net, test_loader, device))
        # epoch의 결과 표시
        print(epoch, train_losses[-1], train_acc[-1],
            val_acc[-1], flush=True)
        if writer is not None:
            writer.add_scalar('train_loss', train_losses[-1], epoch)
            writer.add_scalars('accuracy', {
                "train": train_acc[-1],
                "validation": val_acc[-1]
            }, epoch)


In [17]:
from tensorboardX import SummaryWriter

# SummaryWriter 작성
writer = SummaryWriter("/content/cnn")

# 훈련 실행
net.to("cuda:0")
train_net(net, train_loader, test_loader, n_iter=20, device="cuda:0", writer=writer)


100%|██████████| 469/469 [00:10<00:00, 43.38it/s]


0 0.3256517777967657 0.88215 0.8935999870300293


100%|██████████| 469/469 [00:10<00:00, 42.95it/s]


1 0.287139225910362 0.8945833333333333 0.8965999484062195


100%|██████████| 469/469 [00:10<00:00, 43.04it/s]


...

17 0.15761582151405576 0.94055 0.9218999743461609


100%|██████████| 469/469 [00:11<00:00, 42.63it/s]


18 0.15415703812534484 0.9415 0.9212999939918518


100%|██████████| 469/469 [00:11<00:00, 42.54it/s]


19 0.15295407954507914 0.9423333333333334 0.9168999791145325
